In [13]:
import clickhouse_connect
import polars as pl
import matplotlib.pyplot as plt
import numpy as np
client = clickhouse_connect.get_client(host='hub.publichealthhq.xyz', port=18123, username='default', password='Password123!')
result = client.query("""
SELECT STATE_FIPS, STATE_NAME, COUNTY_FIPS, TOTAL_POPULATION, RUCC
FROM (
    SELECT DISTINCT STATE_NAME, FIPS as COUNTY_FIPS, TOTAL_POPULATION, RUCC
    FROM cps_00004.places_county 
    JOIN cps_00004.rural_urban_codes 
    ON cps_00004.places_county.COUNTY_FIPS = rural_urban_codes.FIPS 
) as inside
JOIN cps_00004.state_fips
ON cps_00004.state_fips.STATE_NAME = inside.STATE_NAME
""")

df = pl.from_dicts(result.named_results(), infer_schema_length=400)

In [14]:
statepop = df.group_by(['STATE_FIPS']).agg( pl.col('TOTAL_POPULATION').sum() )
statepopstep = df.join(statepop, 'STATE_FIPS')\
.group_by(['STATE_FIPS', 'COUNTY_FIPS'])\
.agg( pl.col('TOTAL_POPULATION').sum() / pl.col('TOTAL_POPULATION_right').first() )
statepopstep
stateweighted = statepopstep.join(
    df.select(['STATE_FIPS', 'COUNTY_FIPS', 'RUCC']), on=['STATE_FIPS', 'COUNTY_FIPS'], how='inner')\
.select(['STATE_FIPS', 'COUNTY_FIPS', 'TOTAL_POPULATION', 'RUCC'])

weighted_rucc = stateweighted\
.with_columns( WEIGHTED_RUCC=pl.col('TOTAL_POPULATION') * pl.col('RUCC'))\
.group_by(['STATE_FIPS'])\
.agg( pl.col('WEIGHTED_RUCC').sum()).sort('WEIGHTED_RUCC', descending=True)

In [15]:
weighted_rucc.write_csv('weighted_rucc.csv')

In [7]:
countyRUCC = df.group_by(['STATE_NAME', 'COUNTY_FIPS']).agg( pl.col('RUCC').sum() )
countyRUCC

STATE_NAME,COUNTY_FIPS,RUCC
str,i64,i64
"""Vermont""",50011,3
"""Virginia""",51147,6
"""Texas""",48447,9
"""Virginia""",51169,2
"""Virginia""",51127,1
"""Texas""",48419,7
"""Virginia""",51790,3
"""Virginia""",51067,2
"""Virginia""",51021,8
